In [ ]:
pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

# Function to model Susceptible, Infected, and Recovered populations using a probabilistic infection model
def simulate_infection_graph(population_size, initial_infected, R0, recovery_rate, isolation_rate, days, vaccination_rate, vaccine_efficacy):
    """
    Parameters:
    - population_size: Total population size in the simulation. Affects the overall scale of the outbreak.
    - initial_infected: Initial count of infected individuals at the start. Sets the starting point of the infection curve.
    - R0: Basic reproduction number, representing the average number of people one infected individual will infect if no one is immune.
    - recovery_rate: Rate at which infected individuals recover each day (proportion per day).
    - isolation_rate: Proportion of the population that isolates to reduce infection spread. Lower isolation leads to higher infection spread.
    - days: Duration of the simulation in days.
    - vaccination_rate: Proportion of the population that is vaccinated, which affects susceptibility.
    - vaccine_efficacy: Effectiveness of the vaccine in preventing infection, reducing the infection rate in vaccinated individuals.

    This function simulates and plots the infection dynamics of a population, breaking down vaccinated and unvaccinated groups.
    """

    # Calculate vaccinated and unvaccinated population sizes based on vaccination_rate
    vaccinated_population = population_size * vaccination_rate
    unvaccinated_population = population_size * (1 - vaccination_rate)

    # Initialize arrays to track the susceptible, infected, and recovered populations for vaccinated and unvaccinated groups
    susceptible_vaccinated = [vaccinated_population - initial_infected * vaccination_rate]
    susceptible_unvaccinated = [unvaccinated_population - initial_infected * (1 - vaccination_rate)]

    infected_vaccinated = [initial_infected * vaccination_rate]
    infected_unvaccinated = [initial_infected * (1 - vaccination_rate)]

    recovered_vaccinated = [0]  # Starting with no recovered individuals
    recovered_unvaccinated = [0]

    # Simulate infection dynamics over the specified number of days
    for day in range(1, days + 1):
        # Infection calculation based on R0, population size, and vaccine efficacy
        # Adjust infection rates for vaccinated individuals by efficacy
        new_infected_vaccinated = (infected_vaccinated[-1] * R0 * vaccine_efficacy) * (susceptible_vaccinated[-1] / population_size) * (1 - isolation_rate)
        new_infected_unvaccinated = (infected_unvaccinated[-1] * R0) * (susceptible_unvaccinated[-1] / population_size) * (1 - isolation_rate)

        # Ensure infections do not exceed the number of susceptibles
        new_infected_vaccinated = min(new_infected_vaccinated, susceptible_vaccinated[-1])
        new_infected_unvaccinated = min(new_infected_unvaccinated, susceptible_unvaccinated[-1])

        # Calculate new recoveries based on the current number of infected individuals and recovery rate
        new_recovered_vaccinated = infected_vaccinated[-1] * recovery_rate
        new_recovered_unvaccinated = infected_unvaccinated[-1] * recovery_rate

        # Update the susceptible, infected, and recovered populations
        susceptible_vaccinated.append(susceptible_vaccinated[-1] - new_infected_vaccinated)
        susceptible_unvaccinated.append(susceptible_unvaccinated[-1] - new_infected_unvaccinated)

        infected_vaccinated.append(infected_vaccinated[-1] + new_infected_vaccinated - new_recovered_vaccinated)
        infected_unvaccinated.append(infected_unvaccinated[-1] + new_infected_unvaccinated - new_recovered_unvaccinated)

        recovered_vaccinated.append(recovered_vaccinated[-1] + new_recovered_vaccinated)
        recovered_unvaccinated.append(recovered_unvaccinated[-1] + new_recovered_unvaccinated)

    # Combine vaccinated and unvaccinated populations to get total values
    total_susceptible = np.array(susceptible_vaccinated) + np.array(susceptible_unvaccinated)
    total_infected = np.array(infected_vaccinated) + np.array(infected_unvaccinated)
    total_recovered = np.array(recovered_vaccinated) + np.array(recovered_unvaccinated)

    # Plot the results for susceptible, infected, and recovered populations
    plt.figure(figsize=(10, 6))

    plt.plot(range(days + 1), total_susceptible, label='Susceptible Population', color='orange')
    plt.plot(range(days + 1), total_infected, label='Infected Population', color='red')
    plt.plot(range(days + 1), total_recovered, label='Recovered Population', color='green')

    # Plot enhancements: title, labels, legend, and grid for clarity
    plt.title('Infection Simulation: Susceptible, Infected, and Recovered Populations')
    plt.xlabel('Days')
    plt.ylabel('Population')
    plt.legend()
    plt.grid(True)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Interactive sliders to control the simulation parameters
interact(simulate_infection_graph,
         population_size=IntSlider(min=50, max=500, step=10, value=100, description='Population Size'),
         initial_infected=IntSlider(min=1, max=50, step=1, value=5, description='Initial Infected'),
         R0=FloatSlider(min=0.5, max=5.0, step=0.1, value=2.0, description='R0 (Infection Rate)'),
         recovery_rate=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Recovery Rate'),
         isolation_rate=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, description='Isolation Rate'),
         days=IntSlider(min=1, max=50, step=1, value=20, description='Days'),
         vaccination_rate=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.7, description='Vaccination Rate'),
         vaccine_efficacy=FloatSlider(min=0.5, max=1.0, step=0.01, value=0.9, description='Vaccine Efficacy'));


interactive(children=(IntSlider(value=100, description='Population Size', max=500, min=50, step=10), IntSlider…